# Your very own neural network

In this notebook, we're going to build a neural network using naught but pure numpy and steel nerves. It's going to be fun, I promise!

## Behind the mystery of back-prop

Following the notation of [Michael Nielsen](http://neuralnetworksanddeeplearning.com/chap2.html), we have that

$$\delta^l_j \equiv \frac{\partial C}{\partial z^l_j},$$

where $C$ is the cost and $z^l_j = (\sum_k w^l_{jk}a^{l-1}_k) + b_j^l$, where

$$
w^\text{layer}_{\text{node to layer}, \text{ node from layer - 1}}
$$

We can interpret $\delta^l_j$ as the error from node $j$ in layer $l$, and we will use this to calulate $\frac{\partial C}{\partial w^l_{j,k}}$ and $\frac{\partial C}{\partial b^l_{j}}$ which is used to update the weights and biases in the gradient descent step.

If write out the dependency on the cost, we see that

$$
C = C \Bigg( a^C \bigg(z^C \Big(a^{C-1} \big(z^{C-1} ( ... K) \big) \Big) \bigg) \Bigg)\\
K = a^{l+1} \Bigg( z^{l+1} \bigg( a^l \Big(z^l \big(w^l_{jk}\big) \Big) \bigg) \Bigg)
$$

where we use $K$ as a variable simply for readability.

Using the chain rule (notice the different index on $z$):

$$
\delta^l_j = \sum_k \frac{\partial C}{\partial z^{l+1}_k} \frac{\partial z^{l+1}_k}{\partial z^l_j}\\
= \sum_k \delta^{l+1}_k \frac{\partial z^{l+1}_k}{\partial z^l_j}\\
= \sum_k \delta^{l+1}_k \frac{\partial ([\sum_m w^{l+1}_{km} a^l_m] + b^{l+1}_k)}{\partial z^l_j}\\
= \sum_k \delta^{l+1}_k \frac{\partial ([\sum_m w^{l+1}_{km} \sigma(z^l_m)] + b^{l+1}_k)}{\partial z^l_j}
$$

We see that

$$
\frac{\partial ([\sum_m w^{l+1}_{km} \sigma(z^l_m)] + b^{l+1}_k)}{\partial z^l_j} =
\frac{\partial (w^{l+1}_{kj} \sigma(z^l_j) + b^{l+1}_k)}{\partial z^l_j}
$$

as all other terms are zero, so that

$$
\delta^l_j 
= \sum_k \delta^{l+1}_k \frac{\partial (w^{l+1}_{kj} \sigma(z^l_j) + b^{l+1}_k)}{\partial z^l_j}
= \sum_k \delta^{l+1}_k w^{l+1}_{kj} \frac{\partial \sigma(z^l_j)}{\partial z^l_j}
$$



With this is mind, we can calulate

$$
\frac{\partial C}{\partial b^l_j}
= \sum_k \frac{\partial C}{\partial z^l_{k}} \frac{\partial z^l_{k}}{\partial b^l_j}\\
= \sum_k \delta^l_k \frac{\partial z^l_{k}}{\partial b^l_j}\\
= \sum_k \delta^l_k \frac{\partial ([\sum_m w^l_{km} a^{l-1}_m] + b^l_k)}{\partial b^l_j}\\
= \sum_k \left(\delta^l_k  \frac{\partial (\sum_m w^l_{km} a^{l-1}_m)}{\partial b^l_j} +\delta^l_k \frac{\partial b^l_k}{\partial b^l_j}\right)\\
= \sum_k \delta^l_k \frac{\partial b^l_k}{\partial b^l_j}\\
= \delta^l_j \frac{\partial b^l_j}{\partial b^l_j}\\
= \delta^l_j
$$

and by using the same method as above

$$
\frac{\partial C}{\partial w^l_{no}}
= \sum_k \left(\delta^l_k \frac{\partial (\sum_m w^l_{km} a^{l-1}_m)}{\partial w^l_{no}} + \delta^l_k \frac{\partial b^l_k}{\partial w^l_{no}}\right)\\
=  \sum_k \delta^l_k \frac{\partial (\sum_m w^l_{km} a^{l-1}_m)}{\partial w^l_{no}}\\
= \delta^l_n \frac{\partial (w^l_{no} a^{l-1}_o)}{\partial w^l_{no}}\\
= \delta^l_n a^{l-1}_o
$$

In the setup here, a layer $l$ is split into the calculation of $z$ (dense layer), and then the activation $\sigma(z)$.
$\sigma(z)$ is then passed forward to the next layer $l+1$ in the forward pass.

In the backward pass, the returned variable `grad_input` (input to the previous layer) will refer to two different things.
1. From the activation layer `grad_input` equals $\frac{\partial \sigma(z^l_j)}{\partial z^l_j}$ (input to the dens layer $l$)
2. From the dens layer `grad_input` equals $\frac{\partial C}{\partial z^l_j} \equiv \delta^l_j = \sum_k \delta^{l+1}_k w^{l+1}_{kj} \frac{\partial \sigma(z^l_j)}{\partial z^l_j}$ (input to the previous activation layer $l-1$)

`grad_ouput` (which are the inputs to the dense layer and the activation layer) refers to
1. The gradient from layer $l+1$ in the activation layer (i.e. $\delta^{l+1}$) 
2. The gradient of the activation layer of $l$ in the dense layer (i.e. $\frac{\partial \sigma(z^l_j)}{\partial z^l_j}$) 

### NOTE
The weights of the $l+1$ layer are here stored in the dense layer of layer $l$.

In [ ]:
from pathlib import Path
cur_dir = Path('.').absolute()

src_keras = cur_dir.parent.joinpath('readonly','keras')

if not src_keras.is_dir():
    raise RuntimeError('readonly directory not found\n'
                       'Download it by running download_resources.ipynb in the root dir of the repository')

src_datasets = src_keras.joinpath('datasets')
src_models = src_keras.joinpath('models')

home = cur_dir.home()
dst_datasets = home.joinpath('.keras', 'datasets')
dst_models = home.joinpath('.keras', 'models')
dst_datasets.mkdir(exist_ok=True, parents=True)
dst_models.mkdir(exist_ok=True, parents=True)

In [ ]:
from __future__ import print_function
import time
import numpy as np
np.random.seed(42)

Here goes our main class: a layer that can .forward() and .backward().

In [ ]:
class Layer:
    """
    A building block. Each layer is capable of performing two things:
    
    - Process input to get output:           output = layer.forward(input)
    
    - Propagate gradients through itself:    grad_input = layer.backward(input, grad_output)
    
    Some layers also have learnable parameters which they update during layer.backward.
    """
    def __init__(self):
        """Here you can initialize layer parameters (if any) and auxiliary stuff."""
        # A dummy layer does nothing
        pass
    
    def forward(self, input):
        """
        Takes input data of shape [batch, input_units], returns output data [batch, output_units]
        """
        # A dummy layer just returns whatever it gets as input.
        return input

    def backward(self, input, grad_output):
        """
        Performs a backpropagation step through the layer, with respect to the given input.
        
        To compute loss gradients w.r.t input, you need to apply chain rule (backprop):
        
        d loss / d x  = (d loss / d layer) * (d layer / d x)
        
        Luckily, you already receive d loss / d layer as input, so you only need to multiply it by d layer / d x.
        
        If your layer has parameters (e.g. dense layer), you also need to update them here using d loss / d layer
        """
        # The gradient of a dummy layer is precisely grad_output, but we'll write it more explicitly
        num_units = input.shape[1]
        
        d_layer_d_input = np.eye(num_units)
        
        return np.dot(grad_output, d_layer_d_input) # chain rule

### The road ahead

We're going to build a neural network that classifies MNIST digits. To do so, we'll need a few building blocks:
- Dense layer - a fully-connected layer, $f(X)=W \cdot X + \vec{b}$
- ReLU layer (or any other nonlinearity you want)
- Loss function - crossentropy
- Backprop algorithm - a stochastic gradient descent with backpropageted gradients

Let's approach them one at a time.


### Nonlinearity layer

This is the simplest layer you can get: it simply applies a nonlinearity to each element of your network.

In [ ]:
class ReLU(Layer):
    def __init__(self):
        """ReLU layer simply applies elementwise rectified linear unit to all inputs"""
        pass
    
    def forward(self, input):
        """Apply elementwise ReLU to [batch, input_units] matrix"""
        return np.maximum(0, input)
    
    def backward(self, input, grad_output):
        """Compute gradient of loss w.r.t. ReLU input"""
        relu_grad = input > 0
        # NOTE: Elementwise product
        return grad_output*relu_grad        

In [ ]:
# some tests
from util import eval_numerical_gradient
x = np.linspace(-1,1,10*32).reshape([10,32])
l = ReLU()
grads = l.backward(x,np.ones([10,32])/(32*10))
numeric_grads = eval_numerical_gradient(lambda x: l.forward(x).mean(), x=x)
assert np.allclose(grads, numeric_grads, rtol=1e-3, atol=0),\
    "gradient returned by your layer does not match the numerically computed gradient"

#### Instant primer: lambda functions

In python, you can define functions in one line using the `lambda` syntax: `lambda param1, param2: expression`

For example: `f = lambda x, y: x+y` is equivalent to a normal function:

```
def f(x,y):
    return x+y
```
For more information, click [here](http://www.secnetix.de/olli/Python/lambda_functions.hawk).    

### Dense layer

Now let's build something more complicated. Unlike nonlinearity, a dense layer actually has something to learn.

A dense layer applies affine transformation. In a vectorized form, it can be described as:
$$f(X)= W \cdot X + \vec b $$

Where 
* X is an object-feature matrix of shape [batch_size, num_features],
* W is a weight matrix [num_features, num_outputs] 
* and b is a vector of num_outputs biases.

Both W and b are initialized during layer creation and updated each time backward is called.

In [ ]:
class Dense(Layer):
    def __init__(self, input_units, output_units, learning_rate=0.1, initialization='original'):
        """
        A dense layer is a layer which performs a learned affine transformation:
        f(x) = <W*x> + b
        """
        self.learning_rate = learning_rate
        
        if initialization == 'original':
            # initialize weights with small random numbers. We use normal initialization, 
            # but surely there is something better. Try this once you got it working: http://bit.ly/2vTlmaJ
            self.weights = np.random.randn(input_units, output_units)*0.01
        elif initialization == 'xavier':
            # Initialization based on He et al. 
            # http://arxiv-web3.library.cornell.edu/abs/1502.01852
            # See http://cs231n.github.io/neural-networks-2/#init for further details
            # Note that we also set the bias to zero
            self.weights = np.random.randn(input_units, output_units)*np.sqrt(2/input_units)
            
        self.biases = np.zeros(output_units)
        
    def forward(self,input):
        """
        Perform an affine transformation:
        f(x) = <W*x> + b
        
        input shape: [batch, input_units]
        output shape: [batch, output units]
        """
        
        # NOTE: Numpy has automatic broadcasting when adding a vector to a matrix
        return input @ self.weights + self.biases
    
    def backward(self,input,grad_output):
        """
        Backward propagation of the dense layer
        
        Parameters
        ----------
        input : array, shape (batch, input units)
            The input from the previous activation layer (or network input)
        grad_output : array, shape (batch , output units)
            The gradient from the next activation layer
        
        Returns
        -------
        grad_input : array, shape (batch, input units)
            The gradient from this (dense) layer
        """
        
        # compute d f / d x = d f / d dense * d dense / d x
        # where d dense/ d x = weights transposed
        # NOTE: This is delta^l in the equations above
        grad_input = grad_output @ self.weights.transpose()
        
        # compute gradient w.r.t. weights and biases
        # NOTE: As we are computing w^{l+1} and b^{l+1}, we need to use
        #       \delta^{l+1} = grad_output here
        # NOTE: We are computing THE SUM GRADIENTS OVER THE BATCH, as this
        #       is what we will use in the gradient descent
        #       I.e. we find the gradients of the examples one by one and sum
        #       over all of them

        grad_weights = input.transpose() @ grad_output
        grad_biases = grad_output.sum(axis=0)

        assert grad_weights.shape == self.weights.shape,\
            f'grad_weights.shape: {grad_weights.shape} != self.weights.shape: {self.weights.shape}'
        assert grad_biases.shape == self.biases.shape,\
            f'grad_biases.shape: {grad_biases.shape} != self.biases.shape: {self.biases.shape}'
        # Here we perform a stochastic gradient descent step. 
        # Later on, you can try replacing that with something better.
        self.weights = self.weights - self.learning_rate * grad_weights
        self.biases = self.biases - self.learning_rate * grad_biases
        
        return grad_input

### Testing the dense layer

Here we have a few tests to make sure your dense layer works properly. You can just run them, get 3 "well done"s and forget they ever existed.

... or not get 3 "well done"s and go fix stuff. If that is the case, here are some tips for you:
* Make sure you compute gradients for W and b as __sum of gradients over batch__, not mean over gradients. Grad_output is already divided by batch size.
* If you're debugging, try saving gradients in class fields, like "self.grad_w = grad_w" or print first 3-5 weights. This helps debugging.
* If nothing else helps, try ignoring tests and proceed to network training. If it trains alright, you may be off by something that does not affect network training.

In [ ]:
l = Dense(128, 150)

assert -0.05 < l.weights.mean() < 0.05 and 1e-3 < l.weights.std() < 1e-1,\
    "The initial weights must have zero mean and small variance. "\
    "If you know what you're doing, remove this assertion."
assert -0.05 < l.biases.mean() < 0.05, "Biases must be zero mean. Ignore if you have a reason to do otherwise."

# To test the outputs, we explicitly set weights with fixed values. DO NOT DO THAT IN ACTUAL NETWORK!
l = Dense(3,4)

x = np.linspace(-1,1,2*3).reshape([2,3])
l.weights = np.linspace(-1,1,3*4).reshape([3,4])
l.biases = np.linspace(-1,1,4)

assert np.allclose(l.forward(x),np.array([[ 0.07272727,  0.41212121,  0.75151515,  1.09090909],
                                          [-0.90909091,  0.08484848,  1.07878788,  2.07272727]]))
print("Well done!")

In [ ]:
# To test the grads, we use gradients obtained via finite differences

from util import eval_numerical_gradient

x = np.linspace(-1,1,10*32).reshape([10,32])
l = Dense(32,64,learning_rate=0)

numeric_grads = eval_numerical_gradient(lambda x: l.forward(x).sum(),x)
grads = l.backward(x,np.ones([10,64]))

assert np.allclose(grads,numeric_grads,rtol=1e-3,atol=0), "input gradient does not match numeric grad"
print("Well done!")

In [ ]:
#test gradients w.r.t. params
def compute_out_given_wb(w,b):
    l = Dense(32,64,learning_rate=1)
    l.weights = np.array(w)
    l.biases = np.array(b)
    x = np.linspace(-1,1,10*32).reshape([10,32])
    return l.forward(x)
    
def compute_grad_by_params(w,b):
    l = Dense(32,64,learning_rate=1)
    l.weights = np.array(w)
    l.biases = np.array(b)
    x = np.linspace(-1,1,10*32).reshape([10,32])
    l.backward(x,np.ones([10,64]) / 10.)
    return w - l.weights, b - l.biases
    
w,b = np.random.randn(32,64), np.linspace(-1,1,64)

numeric_dw = eval_numerical_gradient(lambda w: compute_out_given_wb(w,b).mean(0).sum(),w )
numeric_db = eval_numerical_gradient(lambda b: compute_out_given_wb(w,b).mean(0).sum(),b )
grad_w,grad_b = compute_grad_by_params(w,b)

assert np.allclose(numeric_dw,grad_w,rtol=1e-3,atol=0), "weight gradient does not match numeric weight gradient"
assert np.allclose(numeric_db,grad_b,rtol=1e-3,atol=0), "bias gradient does not match numeric bias gradient"
print("Well done!")

### The loss function

Since we want to predict probabilities, it would be logical for us to define softmax nonlinearity on top of our network and compute loss given predicted probabilities. However, there is a better way to do so.

If you write down the expression for crossentropy as a function of softmax logits (a), you'll see:

$$ loss = - log \space {e^{a_{correct}} \over {\underset i \sum e^{a_i} } } $$

If you take a closer look, ya'll see that it can be rewritten as:

$$ loss = - a_{correct} + log {\underset i \sum e^{a_i} } $$

It's called Log-softmax and it's better than naive log(softmax(a)) in all aspects:
* Better numerical stability
* Easier to get derivative right
* Marginally faster to compute

So why not just use log-softmax throughout our computation and never actually bother to estimate probabilities.

Here you are! We've defined the both loss functions for you so that you could focus on neural network part.

In [ ]:
def softmax_crossentropy_with_logits(logits,reference_answers):
    """Compute crossentropy from logits[batch,n_classes] and ids of correct answers"""
    logits_for_answers = logits[np.arange(len(logits)),reference_answers]
    
    xentropy = - logits_for_answers + np.log(np.sum(np.exp(logits),axis=-1))
    
    return xentropy

def grad_softmax_crossentropy_with_logits(logits,reference_answers):
    """Compute crossentropy gradient from logits[batch,n_classes] and ids of correct answers"""
    ones_for_answers = np.zeros_like(logits)
    ones_for_answers[np.arange(len(logits)),reference_answers] = 1
    
    softmax = np.exp(logits) / np.exp(logits).sum(axis=-1,keepdims=True)
    
    return (- ones_for_answers + softmax) / logits.shape[0]

In [ ]:
logits = np.linspace(-1,1,500).reshape([50,10])
answers = np.arange(50)%10

softmax_crossentropy_with_logits(logits,answers)
grads = grad_softmax_crossentropy_with_logits(logits,answers)
numeric_grads = eval_numerical_gradient(lambda l: softmax_crossentropy_with_logits(l,answers).mean(),logits)

assert np.allclose(numeric_grads,grads,rtol=1e-3,atol=0), "The reference implementation has just failed. Someone has just changed the rules of math."

### Full network

Now let's combine what we've just built into a working neural network. As we announced, we're gonna use this monster to classify handwritten digits, so let's get them loaded.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

from preprocessed_mnist import load_dataset
X_train, y_train, X_val, y_val, X_test, y_test = load_dataset(flatten=True)

plt.figure(figsize=[6,6])
for i in range(4):
    plt.subplot(2,2,i+1)
    plt.title("Label: %i"%y_train[i])
    plt.imshow(X_train[i].reshape([28,28]),cmap='gray');

We'll define network as a list of layers, each applied on top of previous one. In this setting, computing predictions and training becomes trivial.

In [ ]:
network = []
network.append(Dense(X_train.shape[1],100))
network.append(ReLU())
network.append(Dense(100,200))
network.append(ReLU())
network.append(Dense(200,10))

In [ ]:
def forward(network, X):
    """
    Compute activations of all network layers by applying them sequentially.
    Return a list of activations for each layer. 
    Make sure last activation corresponds to network logits.
    """
    activations = []
    input = X

    # <your code here>
    for layer in network:
        # Pass the input through the layers
        input = layer.forward(input)
        activations.append(input)
        
    assert len(activations) == len(network), \
        f'len(activations): {len(activations)} != len(network): {len(network)}'
    return activations

In [ ]:
def predict(network,X):
    """
    Compute network predictions.
    """
    logits = forward(network,X)[-1]
    return logits.argmax(axis=-1)

In [ ]:
def train(network,X,y):
    """
    Train your network on a given batch of X and y.
    You first need to run forward to get all layer activations.
    Then you can run layer.backward going from last to first layer.
    
    After you called backward for all layers, all Dense layers have already made one gradient step.
    """
    
    # Get the layer activations
    layer_activations = forward(network,X)
    # NOTE: Concatenation of lists
    layer_inputs = [X]+layer_activations  #layer_input[i] is an input for network[i]
    logits = layer_activations[-1]
    
    # Compute the loss and the initial gradient
    loss = softmax_crossentropy_with_logits(logits,y)
    loss_grad = grad_softmax_crossentropy_with_logits(logits,y)
    
    # <your code: propagate gradients through the network>
    grad_output = loss_grad
    # NOTE: The layers are structured as follows
    #       Dense_0, Activation_0, Dense_1, Activation_1, ..., Dense_n, softmax
    #       The calculation of the loss and the gradient of the loss has
    #       been calculated for us above
    #       By looping backwords as we do it below we get
    #       Dense_n, ..., Activation_1, Dense_1, Activation_0, Dense_0
    #       i.e. without the softmax part
    for i in range(len(network))[::-1]:
        grad_output = network[i].backward(layer_inputs[i],grad_output)
    
    return np.mean(loss)

Instead of tests, we provide you with a training loop that prints training and validation accuracies on every epoch.

If your implementation of forward and backward are correct, your accuracy should grow from 90~93% to >97% with the default network.

### Training loop

As usual, we split data into minibatches, feed each such minibatch into the network and update weights.

In [ ]:
from tqdm import trange
def iterate_minibatches(inputs, targets, batchsize, shuffle=False):
    assert len(inputs) == len(targets)
    if shuffle:
        indices = np.random.permutation(len(inputs))
    for start_idx in trange(0, len(inputs) - batchsize + 1, batchsize):
        if shuffle:
            excerpt = indices[start_idx:start_idx + batchsize]
        else:
            excerpt = slice(start_idx, start_idx + batchsize)
        yield inputs[excerpt], targets[excerpt]

In [ ]:
# NOTE: Casted this to a function for simpler reuse
from IPython.display import clear_output

def execute_train(network, epochs=25):
    train_log = []
    val_log = []
    for epoch in range(25):

        for x_batch,y_batch in iterate_minibatches(X_train,y_train,batchsize=32,shuffle=True):
            train(network,x_batch,y_batch)

        train_log.append(np.mean(predict(network,X_train)==y_train))
        val_log.append(np.mean(predict(network,X_val)==y_val))

        clear_output()
        print("Epoch",epoch)
        print("Train accuracy:",train_log[-1])
        print("Val accuracy:",val_log[-1])
        plt.plot(train_log,label='train accuracy')
        plt.plot(val_log,label='val accuracy')
        plt.legend(loc='best')
        plt.grid()
        plt.show()

In [ ]:
execute_train(network, epochs=25)

### Peer-reviewed assignment

Congradulations, you managed to get this far! There is just one quest left undone, and this time you'll get to choose what to do.


#### Option I: initialization
* Implement Dense layer with Xavier initialization as explained [here](http://bit.ly/2vTlmaJ)

To pass this assignment, you must conduct an experiment showing how xavier initialization compares to default initialization on deep networks (5+ layers).


#### Option II: regularization
* Implement a version of Dense layer with L2 regularization penalty: when updating Dense Layer weights, adjust gradients to minimize

$$ Loss = Crossentropy + \alpha \cdot \underset i \sum {w_i}^2 $$

To pass this assignment, you must conduct an experiment showing if regularization mitigates overfitting in case of abundantly large number of neurons. Consider tuning $\alpha$ for better results.

#### Option III: optimization
* Implement a version of Dense layer that uses momentum/rmsprop or whatever method worked best for you last time.

Most of those methods require persistent parameters like momentum direction or moving average grad norm, but you can easily store those params inside your layers.

To pass this assignment, you must conduct an experiment showing how your chosen method performs compared to vanilla SGD.

### General remarks
_Please read the peer-review guidelines before starting this part of the assignment._

In short, a good solution is one that:
* is based on this notebook
* runs in the default course environment with Run All
* its code doesn't cause spontaneous eye bleeding
* its report is easy to read.

_Formally we can't ban you from writing boring reports, but if you bored your reviewer to death, there's noone left alive to give you the grade you want._

## Option I: Initialization

I chose this task, as this is the one I know the least about, and I would like to know some more.
Instead of duplicate the code for the dense layers, I've simply put a flag in the `Dense`-class which defaults to the original initialization.

We will use the initialization method proposed by [He et al.](http://arxiv-web3.library.cornell.edu/abs/1502.01852) as we will use ReLu's as activation functions (see also this [link](http://cs231n.github.io/neural-networks-2/#init) for details, which you can find references to several places on the internet).

We will compare training the same architecture with and without the Xavier initialization

In [ ]:
def get_network(initialization):
    network = []
    network.append(Dense(X_train.shape[1],100, initialization=initialization))
    network.append(ReLU())
    network.append(Dense(100,100, initialization=initialization))
    network.append(ReLU())
    network.append(Dense(100,100, initialization=initialization))
    network.append(ReLU())
    network.append(Dense(100,100, initialization=initialization))
    network.append(ReLU())
    network.append(Dense(100,100, initialization=initialization))
    network.append(ReLU())
    network.append(Dense(100,10, initialization=initialization))

    return network

### Original

In [ ]:
t = time.time()
network = get_network('original')
execute_train(network, epochs=25)
print(f'elapsed = {(time.time() - t)/60:.2f} minutes')

### Xavier

In [ ]:
t = time.time()
network = get_network('xavier')
execute_train(network, epochs=25)
print(f'elapsed = {(time.time() - t)/60:.2f} minutes')

### Conclusion

From the runs above we can see that the network with the original initialization performs approximately like random guessing and seems not to improve the accuracy as we train the network.

This, however, is not the problem for the network using the He variant of the Xavier initialization. As explained [here](http://andyljones.tumblr.com/post/110998971763/an-explanation-of-xavier-initialization), this due to the possibility of the signal to propagate to the network without being damped or significantly being enhanced rigth after the initialization. This means that the signal can reach the end of the network so that the backpropagation is the sole tuner of which signals should be damped/enhanced (and not the initialization)

## Bonus assignments

As a bonus assignment (no points, just swag), consider implementing Batch Normalization ([guide](https://gab41.lab41.org/batch-normalization-what-the-hey-d480039a9e3b)) or Dropout ([guide](https://medium.com/@amarbudhiraja/https-medium-com-amarbudhiraja-learning-less-to-learn-better-dropout-in-deep-machine-learning-74334da4bfc5)). Note, however, that those "layers" behave differently when training and when predicting on test set.

* Dropout:
  * During training: drop units randomly with probability __p__ and multiply everything by __1/(1-p)__
  * During final predicton: do nothing; pretend there's no dropout
  
* Batch normalization
  * During training, it substracts mean-over-batch and divides by std-over-batch and updates mean and variance.
  * During final prediction, it uses accumulated mean and variance.